In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()



# 그래프 기본 설정
#plt.rcParams['font.family'] = 'NanumGothic' #리눅스용
plt.rcParams['font.family'] = 'Malgun Gothic' #윈도우용
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False


#label_encoder
from sklearn.preprocessing import LabelEncoder

#다중공산성을 봐봅시다
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

#파일 경로
file_dir = "C:/Users/sherm/Desktop/workspace/likeLion_final_project/data/train/0.종합본/"

In [2]:
df=pd.read_parquet(file_dir + "train_잔액정보_append.parquet")

#### 비어있는 값들부터 drop

In [4]:
df_temp=df.drop(['기준년월','ID'],axis=1)

In [5]:
drop_list=['연체일자_B0M','카드론잔액_최종경과월','최종연체개월수_R15M','RV잔액이월횟수_R3M','RV잔액이월횟수_R6M','연체잔액_일시불_해외_B0M','연체잔액_CA_해외_B0M']
df_temp.drop(drop_list, axis=1,inplace=True)
df_temp

,잔액_일시불_B0M,잔액_할부_B0M,잔액_현금서비스_B0M,잔액_리볼빙일시불이월_B0M,잔액_리볼빙CA이월_B0M,잔액_카드론_B0M,월중평잔_일시불_B0M,월중평잔_할부_B0M,월중평잔_CA_B0M,연체잔액_B0M,...,평잔_일시불_6M,평잔_일시불_해외_6M,평잔_RV일시불_6M,평잔_RV일시불_해외_6M,평잔_할부_6M,평잔_할부_해외_6M,평잔_CA_6M,평잔_CA_해외_6M,평잔_카드론_6M,Segment
0,998,962,22971,0,0,0,1084,547,25032,0,...,2440,0,0,0,572,0,17008,0,0,D
1,998,962,22971,0,0,0,1084,547,25032,0,...,2440,0,0,0,572,0,17008,0,0,D
2,998,962,22971,0,0,0,1084,547,25032,0,...,2440,0,0,0,572,0,17008,0,0,D
3,2565,2390,0,0,0,0,4090,2553,0,0,...,2677,0,2830,0,2736,0,0,0,0,E
4,2565,2390,0,0,0,0,4090,2553,0,0,...,2677,0,2830,0,2736,0,0,0,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4799995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
4799996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
4799997,1110,4093,0,0,0,0,637,2780,0,0,...,2415,0,0,0,6558,0,0,0,0,E
4799998,1110,4093,0,0,0,0,637,2780,0,0,...,2415,0,0,0,6558,0,0,0,0,E


#### 파생변수 생성

In [10]:
df_temp['diff_연체원금_B1-2M'] = df_temp['연체원금_B1M'] - df_temp['연체원금_B2M']

conditions = [
    df_temp['diff_연체원금_B1-2M'] > 0,
    df_temp['diff_연체원금_B1-2M'] < 0,
    df_temp['diff_연체원금_B1-2M'] == 0
]

choices = ['증가', '감소', '유지']

df_temp['diff_연체원금_B1-2M_category'] = np.select(conditions, choices, default='기타')

In [8]:
df_temp['연체잔액_총합_B0M'] = df_temp['연체잔액_일시불_B0M'] + df_temp['연체잔액_할부_B0M'] + df_temp['연체잔액_현금서비스_B0M'] + df_temp['연체잔액_카드론_B0M'] + df_temp['연체잔액_대환론_B0M']

In [9]:
df_temp["잔액_총합_B0M"]=df_temp["잔액_일시불_B0M"]+df_temp["잔액_할부_B0M"]+df_temp['잔액_현금서비스_B0M']

In [11]:
df_temp['리볼빙CA_그룹'] = pd.cut(
    df_temp['잔액_리볼빙CA이월_B0M'],
    bins=[-1, 1, 1000, 3000, 4000, 10000, 14000, 20000, 30000, 50000],
    labels=False
)
df_temp['리볼빙CA_그룹'].value_counts()

리볼빙CA_그룹
0    4797114
7       1681
4        621
5        369
6        134
2         45
8         24
3         11
1          1
Name: count, dtype: int64

In [13]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4800000 entries, 0 to 4799999
Data columns (total 79 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   잔액_일시불_B0M                int64 
 1   잔액_할부_B0M                 int64 
 2   잔액_현금서비스_B0M              int64 
 3   잔액_리볼빙일시불이월_B0M           int64 
 4   잔액_리볼빙CA이월_B0M            int64 
 5   잔액_카드론_B0M                int64 
 6   월중평잔_일시불_B0M              int64 
 7   월중평잔_할부_B0M               int64 
 8   월중평잔_CA_B0M               int64 
 9   연체잔액_B0M                  int64 
 10  연체잔액_일시불_B0M              int64 
 11  연체잔액_할부_B0M               int64 
 12  연체잔액_현금서비스_B0M            int64 
 13  연체잔액_카드론_B0M              int64 
 14  연체잔액_대환론_B0M              int64 
 15  잔액_현금서비스_B1M              int64 
 16  잔액_현금서비스_B2M              int64 
 17  잔액_카드론_B1M                int64 
 18  잔액_카드론_B2M                int64 
 19  잔액_카드론_B3M                int64 
 20  잔액_카드론_B4M                int64 
 21  잔액_카드론_B

In [14]:
연체원금_diff_encoder = LabelEncoder()
Segment_encoder = LabelEncoder()

df_temp['diff_연체원금_B1-2M_category']=연체원금_diff_encoder.fit_transform(df_temp['diff_연체원금_B1-2M_category'])
df_temp['Segment']=연체원금_diff_encoder.fit_transform(df_temp['Segment'])

In [15]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4800000 entries, 0 to 4799999
Data columns (total 79 columns):
 #   Column                    Dtype
---  ------                    -----
 0   잔액_일시불_B0M                int64
 1   잔액_할부_B0M                 int64
 2   잔액_현금서비스_B0M              int64
 3   잔액_리볼빙일시불이월_B0M           int64
 4   잔액_리볼빙CA이월_B0M            int64
 5   잔액_카드론_B0M                int64
 6   월중평잔_일시불_B0M              int64
 7   월중평잔_할부_B0M               int64
 8   월중평잔_CA_B0M               int64
 9   연체잔액_B0M                  int64
 10  연체잔액_일시불_B0M              int64
 11  연체잔액_할부_B0M               int64
 12  연체잔액_현금서비스_B0M            int64
 13  연체잔액_카드론_B0M              int64
 14  연체잔액_대환론_B0M              int64
 15  잔액_현금서비스_B1M              int64
 16  잔액_현금서비스_B2M              int64
 17  잔액_카드론_B1M                int64
 18  잔액_카드론_B2M                int64
 19  잔액_카드론_B3M                int64
 20  잔액_카드론_B4M                int64
 21  잔액_카드론_B5M                int64

In [16]:
X = df_temp.drop('Segment',axis=1)
X_const = add_constant(X)
vif_df_01 = pd.DataFrame()
vif_df_01["feature"] = X.columns
vif_df_01["VIF"] = [variance_inflation_factor(X_const.values, i+1) for i in range(len(X.columns))]

In [19]:
vif_df_01.nlargest(50,'VIF')

,feature,VIF
0,잔액_일시불_B0M,inf
1,잔액_할부_B0M,inf
2,잔액_현금서비스_B0M,inf
11,연체잔액_할부_B0M,inf
12,연체잔액_현금서비스_B0M,inf
13,연체잔액_카드론_B0M,inf
14,연체잔액_대환론_B0M,inf
28,연체원금_B1M,inf
29,연체원금_B2M,inf
73,연체잔액_총합_B0M,inf
